In [10]:
import requests # for web scraping
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import numpy as np

# sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
#from sentiment_dictionary import NEW_WORDS

# install yahoo finance
import yfinance as yf

from config import save_path

In [2]:
def scrape_finviz(companies):
    """A function to scrape the finviz website.
    
    It will return the information as a data frame with columns for:
    company, date, time, headline.
    """
    
    news_tables = {} # dictionary to store the news tables from each company
    # url for the news website
    finwiz_url = 'https://finviz.com/quote.ashx?t='
    
    print('{} companies to gather data for'.format(len(companies)))
    count = 1
    
    # loop through each company you want to store data for
    for company in companies:
        print(count)
        try:
            url = finwiz_url + company
            response = requests.get(url, headers={'user-agent': 'my-app/0.0.1'})
            soup = BeautifulSoup(response.text, 'html.parser')
            # find 'news-table' in soup and load it into 'news_tables'
            news_table = soup.find(id='news-table')
            # add the table to dictionary
            news_tables[company] = news_table
            count+=1
        except:
            print('error with {}'.format(company))
            count +=1
            continue
        
    parsed_news = []
    # iterate through the news
    # iterate through the news
    for file_name, news_table in news_tables.items():
        try:
            # iterate through all tr tags in 'news_table'
            for article in news_table.find_all('tr'):
                # read the text from each tr tag into text
                # get text from a only
                text = article.a.get_text()
                # splice text in the td tag into a list
                date_scrape = article.td.text.split()
                # if the length of the 'date_scrape' is 1, load 'time' as the only element

                if len(date_scrape) == 1:
                    time = date_scrape[0]

                # else load 'date as the 1st element and 'time' as the second
                else:
                    date = date_scrape[0]
                    time = date_scrape[1]
                # extract the ticker from the file name, get the string up to the first '_'
                company = file_name.split('_')[0]

                # append ticker, date, time, and headline as a list to the parsed news list
                parsed_news.append([company, date, time, text])
        except:
            print('error with {}'.format(file_name.split('_')[0]))
            continue
            
    # convert parsed news list into a dataframe 
    columns = ['company','date','time','headline']
    parsed_news = pd.DataFrame(parsed_news, columns = columns)
    
    # delete news from Saturday and Sunday
    index_names = parsed_news[pd.to_datetime(parsed_news['date']).dt.weekday >=5].index
    parsed_news.drop(index_names, inplace = True)
    
    # reset index
    parsed_news = parsed_news.reset_index(drop=True)
    
    return parsed_news

In [3]:
scrape_finviz(['AAPL'])

1 companies to gather data for
1
error with AAPL


NameError: name 'pd' is not defined